Dit notebook heeft als doel om NER-herkende verwijzingen naar hoedanigheden te verrijken met standaardnamen. Met hoedanigheden bedoelen we titels, rollen, en functies, zoals "keizer" en "ambassadeur", maar ook "weduwe" en "moeder". Dit doen we door de volgende stappen toe te passen:

- DataFrame Aanmaken: We creëren een DataFrame met geaggregeerde verwijzingen en annotatiekolommen.
- Thesaurus Toepassen: We gebruiken een thesaurus met reguliere expressies om hoedanigheidstags toe te voegen aan de opgeschoonde strings.
- Hoedanigheden Herkennen: We zoeken ook naar andere hoedanigheden in delen van de string vóór en na de gevonden hoedanigheid.
- Gegevens Samenvoegen: We voegen de geannoteerde gegevens samen met het annotatiebestand en verwijderen ongewenste kolommen.
- Resultaten Controleren: We berekenen het percentage van de regels met niet-lege annotaties.

Met deze stappen zorgen we ervoor dat alle annotaties en relevante informatie correct zijn gekoppeld en weergegeven in het eindresultaat.

In [ ]:
%reload_ext autoreload
%autoreload 2
import os
import sys
import re
from collections import Counter, defaultdict
import json
from tqdm.notebook import tqdm
from datetime import datetime
# import republic.model.republic_document_model as rdm
import pandas as pd
from substitutions import substitutions
from deletions import deletions
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
repo_dir = os.path.split(os.getcwd())[0]
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

Stap 1: Aanmaken en opschonen df

In deze eerste stap creëren we een DataFrame met de annotaties. Dit DataFrame bevat:

- Geaggregeerde verwijzingen: Alle verwijzingen zijn samengevoegd in één DataFrame. Met verwijzingen bedoelen we de output van de NER-tagger, dus de entiteiten zoals ze in de tekst zijn herkend.
- Annotatiekolommen: De kolommen anno_name1 tot en met anno_name5 zijn toegevoegd om annotaties (hoedanigheden-tags) te registreren.
- Subframes: We hebben regels opgesteld om subframes te herkennen, zoals "df_delete" en "df_unmatched", om te kunnen meten hoeveel rijen er al wel of niet thuisgebracht zijn.

Met de stap df = df[df['aantal'] >= 10] kunnen we kiezen om te werken met het hele DataFrame, inclusief strings die maar één keer voorkomen, of om te beginnen met vaker voorkomende strings. 


In [ ]:
annotations = pd.read_csv('./annotations-layer-HOE.tsv', sep='\t')
annotations['tag_new'] = annotations['tag_text'].apply(lambda x: re.sub(r'^[^a-zA-Z0-9]+', '', x))

df_temp = annotations.groupby('tag_new').size().reset_index(name='aantal')
column_names = ['tag', 'lowertag', 'aantal', 'anno_name1', 'anno_name2', 'anno_name3',
                'anno_name4', 'anno_name5', 'delete', 'best_score', 'select', 'uitzoeken/onduidelijk']
df = pd.DataFrame(columns=column_names)
df['tag'] = df_temp['tag_new']
df['aantal'] = df_temp['aantal']
df = df[df['aantal'] >= 5] #We doen het nu even meet alles wat meer dan 10 keer voorkomt, om sneller overal doorheen te lopen
print(f"Dataframe bestaat uit {len(df)} rijen")
anno_columns = ['anno_name1', 'anno_name2', 'anno_name3', 'anno_name4', 'anno_name5']
df_delete = df[df['delete'] == 'x']
df_zonder_delete = df[~(df['delete'] == 'x')]
df_matched = df_zonder_delete[df_zonder_delete[['anno_name1', 'anno_name2', 'anno_name3', 'anno_name4']].notna().any(axis=1)]
df_unmatched = df_zonder_delete[df_zonder_delete[anno_columns].isna().all(axis=1) & ~df['delete'].astype(str).eq('x')]

df['lowertag'] = df['tag'].str.lower()
for column in anno_columns:
    if df[column].dtype != 'object':  
        df[column] = df[column].astype(str)

We maken een gestandaardiseerde versie van de hoedanigheden aan, waarbij spellingsnormalisatie is toegepast. Dit passen we alleen toe op de "lowertag", dat wil zeggen, een kopie van de string die in kleine letters is. De oorspronkelijke strings blijven staan. Door alles in kleine letters te zetten, zijn de tags niet hoofdlettergevoelig. Dit omdat hoedanigheden in het corpus geen standaardregels gebruiken voor capitalisering.

De vervangingen in "substitutions.py" zijn grofweg onder te verdelen in:
- leestekens: standaardiseren van komma's, spaties, en verwijderen van onnodige leestekens
- Veelvoorkomende hoedanigheden: standaardiseren van woorden als "admiraal", "ambassadeur", "baron", "keurvorst" e.d. om (delen van) strings makkelijker aan hoedanigheden te matchen
- Spelling: Om spellingsvariatie te verminderen en matchen makkelijker te maken, vervangen we "ae" door "aa", "gt" door "cht" en "uij" of "uy" door "ui".
- We hebben ook de verschillende verwijzingen naar afkomst, geboorteplaats en woonplaats gestandaardiseerd, zo wordt bijvoorbeeld "inwoonder van" en "woonende te" allebei "woonachtig in". 

In [ ]:
from substitutions import substitutions

def apply_substitutions(df):
    substitution_count = 0 
    for pattern, replacement in tqdm(substitutions.items(), desc="Vervangingen maken"):
        try:
            temp = df['lowertag'].str.count(pattern, flags=re.IGNORECASE) 
            substitution_count += temp.sum()
            df['lowertag'] = df['lowertag'].str.replace(pattern, replacement, regex=True)
        except Exception as e:
            print(f"Error occurred with pattern: {pattern}")
            print(e)
    
    print(f"Klaar! {substitution_count} vervangingen gemaakt")
    return df 

df = apply_substitutions(df)
df.loc[df['lowertag'] == '', 'delete'] = 'x'
df_delete = df[df['delete'] == 'x']
df_zonder_delete = df[~(df['delete'] == 'x')]
df_matched = df_zonder_delete[df_zonder_delete[['anno_name1', 'anno_name2', 'anno_name3', 'anno_name4']].notna().any(axis=1)]
df_unmatched = df_zonder_delete[df_zonder_delete[anno_columns].isna().all(axis=1) & ~df['delete'].astype(str).eq('x')]


Met de dictionary deletions verwijderen we eigennamen en overbodige informatie die ons niet helpt bij het vinden van hoedanigheden. Deze opschoning gebeurt in drie stappen:

- Onnodige leestekens en spaties verwijderen: We verwijderen leestekens zoals @/#=„.:,?!"¬,;:= en overbodige spaties.
- Namen verwijderen: We verwijderen namen op basis van de standaardformule "de heeren van ... tot ..." en een lijst van veelvoorkomende namen (namen_list) uit andere bronnen. We richten ons alleen op functies, en hoe korter en eenvoudiger de strings, hoe gemakkelijker het is om hoedanigheden te herkennen.
- Filler words verwijderen: We verwijderen datums, telwoorden, en woorden zoals "wijlen", "gewesen", "geweldigen", enzovoort, omdat deze ons niet verder helpen.

Deze stappen maken de zoekstrings korter en eenvoudiger, wat het proces van het vinden van hoedanigheden efficiënter maakt.



In [ ]:
from deletions import deletions

def apply_deletions(df):
    deletion_count = 0 
    for pattern, replacement in tqdm(deletions.items(), desc="Rommel verwijderen"):
        try:
            temp = df['lowertag'].str.count(pattern, flags=re.IGNORECASE) 
            deletion_count += temp.sum()
            df['lowertag'] = df['lowertag'].str.replace(pattern, replacement, regex=True)
        except Exception as e:
            print(f"Error occurred with pattern: {pattern}")
            print(e)
    
    print(f"Klaar! {deletion_count} verwijderingen gemaakt")
    return df 

df = apply_deletions(df)
df.loc[df['lowertag'] == '', 'delete'] = 'x'
df_delete = df[df['delete'] == 'x']
anno_columns = ['anno_name1', 'anno_name2', 'anno_name3', 'anno_name4', 'anno_name5']
df_zonder_delete = df[~(df['delete'] == 'x')]
df_matched = df_zonder_delete[df_zonder_delete[['anno_name1', 'anno_name2', 'anno_name3', 'anno_name4']].notna().any(axis=1)]
df_unmatched = df_zonder_delete[df_zonder_delete[anno_columns].isna().all(axis=1) & ~df['delete'].astype(str).eq('x')]
print(f"nog niet gematcht: {len(df_unmatched)}")
#df.to_pickle('df_basestaart.pkl') #evt opslaan van dataframe. 

Stap 2: Koppelen hoedanigheden

De belangrijkste stap in dit notebook is de koppeling: het toepassen van de thesaurus van hoedanigheden op de opgeschoonde strings (lowertag). Dit houdt in dat we hoedanigheidstags toekennen aan verwijzingen uit de tagger, waarbij de opgeschoonde versie als tussenstap dient. Bijvoorbeeld, "extraord. envoyé aen het Hof van den Keijzer" krijgt als 'lowertag' "extraordinaris envoyé aan het hof van de keizer", met de annotatie-tags (anno_name) "extraordinaris envoyé" en "keizer".

De thesaurus is bottom-up opgebouwd uit het materiaal zelf. We ontdekten dat hoedanigheden vaak worden ingeleid door woorden zoals "zijnde" of "in leven", of door een komma. We genereerden een lijst van veelvoorkomende unieke woorden die deze patronen volgen en schreven reguliere expressies om verschillende variaties te dekken. Dit resulteerde in het kws_hoe dictionary, dat bestaat uit reguliere expressies en hun bijbehorende tags, bijvoorbeeld: \\bextraor\\w* *[ie][mn].{0,3}[ijy]+\\w*': 'extraordinaris envoyé'.

We pasten deze reguliere expressies steeds toe op df_unmatched['lowertag']. Bij een match wordt de tag toegevoegd aan de eerstvolgende beschikbare anno_name kolom, beginnend met anno_name1, waardoor de rij wordt toegevoegd aan df_matched. Het doel is om bij iedere iteratie de thesaurus uit te breiden en het aantal niet-gematchte verwijzingen te verminderen. Zodra het aantal niet-gematchte strings nihil was, verdeelden we kws_hoe grofweg in categorieën. Dit stelt ons in staat om de herkenning van entiteiten in stappen uit te voeren. 


De cel hieronder is alleen nodig als we hier zouden beginnen, dus met een opgeschoond dataframe van een vorige sessie werken.

In [ ]:
#df = pd.read_pickle('df_base3.pkl')
df.loc[df['lowertag'] == '', 'delete'] = 'x'
df_delete = df[df['delete'] == 'x']
anno_columns = ['anno_name1', 'anno_name2', 'anno_name3', 'anno_name4', 'anno_name5']
df_zonder_delete = df[~(df['delete'] == 'x')]
df_matched = df_zonder_delete[df_zonder_delete[['anno_name1', 'anno_name2', 'anno_name3', 'anno_name4']].notna().any(axis=1)]
df_unmatched = df_zonder_delete[df_zonder_delete[anno_columns].isna().all(axis=1) & ~df['delete'].astype(str).eq('x')]
len(df_unmatched)

In deze stap passen we annotaties toe op de opgeschoonde gegevens met behulp van reguliere expressies uit kw_dict. Het doel is om relevante tags te identificeren en aan de data toe te voegen.

We proberen elk regex-patroon uit de patterns-lijst op de lowertag toe te passen.Als een patroon een match vindt, wordt de string opgedeeld in drie delen: vóór de match [0], de match zelf [1], en na de match [2]. Deze delen worden samen met de overeenkomstige tag en het originele patroon toegevoegd aan een tijdelijk resultaat DataFrame ["new_df"]. We bewaren het originele regex patroon in de kolom provenance. 


In [ ]:
def apply_annotations(df, kw_dict):
    patterns = []

    # Compile regex patterns from the dictionary
    for pat, repl in kw_dict.items():
        try:
            pattern_with_group = f"(.*) *(\\b{pat}\\b) *(.*)"
            ptrn = re.compile(pattern_with_group, flags=re.I)
            patterns.append((ptrn, pat, repl))  # Store the pattern, original 'pat', and replacement
        except re.error as e:
            print(f"Error met het patroon '{pattern_with_group}': {e}")

    results = []

    # Iterate over each row in the DataFrame with a progress bar
    for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
        lowertag = row['lowertag']
        aantal = row['aantal']  # Preserve the 'aantal' value
        matched = False

        for pattern, pat, replacement in patterns:
            if matched:
                break

            match = pattern.match(lowertag)
            if match:
                matched = True
                matches = pd.DataFrame({
                    0: [match.group(1)],
                    1: [match.group(2)],
                    2: [match.group(3)],
                    'provenance': [pat],  # Use the original 'pat' for provenance
                    'anno_name1': [replacement],  # Use the 'replacement' from kw_dict
                    'aantal': [aantal]  # Add the 'aantal' column to the result
                }, index=[idx])
                results.append(matches)

    if results:
        offpat = pd.concat(results)
        offpat = offpat.sort_values('provenance')
    else:
        offpat = pd.DataFrame(columns=[0, 1, 2, 'provenance', 'anno_name1', 'aantal'])

    return offpat


Nu lopen we de verschillende categorieën van de hoedanigheden door.

In [ ]:
from kws_hoe import kw_edelen
offpat = apply_annotations(df_unmatched, kw_edelen)
new_df = offpat
common_indices = new_df.index.get_level_values(0).intersection(df_unmatched.index)
df_unmatched = df_unmatched.drop(common_indices)
print(f"gematcht: {len(new_df)}")
print(f"nog niet gematcht: {len(df_unmatched)}")
if new_df.index.has_duplicates:
    print("Let op! New_df heeft duplicate labels")

In [ ]:
from kws_hoe import kw_kerk
offpat = apply_annotations(df_unmatched, kw_kerk)
new_df = pd.concat([new_df, offpat])
common_indices = new_df.index.get_level_values(0).intersection(df_unmatched.index)
df_unmatched = df_unmatched.drop(common_indices)

print(f"gematcht: {len(new_df)}")
print(f"nog niet gematcht: {len(df_unmatched)}")
if new_df.index.has_duplicates:
    print("Let op! New_df heeft duplicate labels")


In [ ]:
from kws_hoe import kw_ambt
offpat = apply_annotations(df_unmatched, kw_ambt)
new_df = pd.concat([new_df, offpat])
common_indices = new_df.index.get_level_values(0).intersection(df_unmatched.index)
df_unmatched = df_unmatched.drop(common_indices)

print(f"gematcht: {len(new_df)}")
print(f"nog niet gematcht: {len(df_unmatched)}")
if new_df.index.has_duplicates:
    print("Let op! New_df heeft duplicate labels")

In [ ]:
from kws_hoe import kw_pol
offpat = apply_annotations(df_unmatched, kw_pol)
new_df = pd.concat([new_df, offpat])
common_indices = new_df.index.get_level_values(0).intersection(df_unmatched.index)
df_unmatched = df_unmatched.drop(common_indices)
print(f"gematcht: {len(new_df)}")
print(f"nog niet gematcht: {len(df_unmatched)}")


In [ ]:
from kws_hoe import kw_id
offpat = apply_annotations(df_unmatched, kw_id)
new_df = pd.concat([new_df, offpat])
common_indices = new_df.index.get_level_values(0).intersection(df_unmatched.index)
df_unmatched = df_unmatched.drop(common_indices)
print(f"gematcht: {len(new_df)}")
print(f"nog niet gematcht: {len(df_unmatched)}")


In [ ]:
from kws_hoe import kw_beroep
#beroepen
offpat = apply_annotations(df_unmatched, kw_beroep)
new_df = pd.concat([new_df, offpat])
common_indices = new_df.index.get_level_values(0).intersection(df_unmatched.index)
df_unmatched = df_unmatched.drop(common_indices)
print(f"gematcht: {len(new_df)}")
print(f"nog niet gematcht: {len(df_unmatched)}")


In [ ]:
from kws_hoe import kw_rest, kw_new
kw_rest.update(kw_new)
offpat = apply_annotations(df_unmatched, kw_rest)
new_df = pd.concat([new_df, offpat])
common_indices = new_df.index.get_level_values(0).intersection(df_unmatched.index)
df_unmatched = df_unmatched.drop(common_indices)
print(f"gematcht: {len(new_df)}")
print(f"nog niet gematcht: {len(df_unmatched)}")


In [ ]:
from kws_hoe import kw_zlast
# dingen die je voor het laatst moet bewaren zodat ze geen verwarring veroorzaken
offpat = apply_annotations(df_unmatched, kw_zlast)
new_df = pd.concat([new_df, offpat])
common_indices = new_df.index.get_level_values(0).intersection(df_unmatched.index)
df_unmatched = df_unmatched.drop(common_indices)
print(f"gematcht: {len(new_df)}")
print(f"nog niet gematcht: {len(df_unmatched)}")


In [ ]:
new_df

In [ ]:
#new_df.to_pickle('new_dftot.pkl') #tussentijds opslaan

Om de strings die geen hoedanigheden zijn effectief uit te sluiten, gebruiken we de patronen in "dellpat" van kws_hoe. Deze patronen helpen ons bij het identificeren van irrelevante strings. De patronen in dellpat omvatten:

- Lege of onbruikbare strings: Bijvoorbeeld '^ *$', wat lege strings identificeert.
- Veelvoorkomende niet-relevante termen: Zoals '^ *somme\b' en '^ *heere?n? *van *\w* *$', die verwijzen naar algemene termen of plaatsnamen.
- Onvolledige of kortere teksten: Bijvoorbeeld '^ *f? *.{0,2} *f? *$', die strings filteren met minder inhoudelijke waarde.

Door deze irrelevante strings uit te sluiten, krijgen we een duidelijker beeld van de resterende gegevens in df_unmatched, zonder de ruis mee te tellen.


In [ ]:
# hele strings wegzetten als zijnde geen hoedanigheid obv dict
from kws_hoe import dellpat
df_todel = df_unmatched[df_unmatched['lowertag'].str.contains('|'.join(dellpat), regex=True)]
df_todel['delete']='x'
len(df_todel)
df.update(df_todel)
df_delete = df[df['delete'] == 'x']
df_zonder_delete = df[~(df['delete'] == 'x')]
df_matched = df_zonder_delete[df_zonder_delete[['anno_name1', 'anno_name2', 'anno_name3', 'anno_name4']].notna().any(axis=1)]
df_unmatched = df_zonder_delete[df_zonder_delete[anno_columns].isna().all(axis=1) & ~df['delete'].astype(str).eq('x')]
common_indices = new_df.index.get_level_values(0).intersection(df_unmatched.index)
df_unmatched = df_unmatched.drop(common_indices)
print(f"gematcht: {len(new_df)}")
print(f"nog niet gematcht: {len(df_unmatched)}")


**Stap 3**: Na het herkennen van een hoedanigheid kunnen er vaak nog andere hoedanigheden in dezelfde string staan, zowel voor als na de herkende hoedanigheid. Daarom zoeken we ook in de delen van de string vóór en ná de gevonden hoedanigheid (in het resultatenframe heten deze delen respectievelijk "0" en "2").

Bijvoorbeeld, in de string "extraord. envoyé aen het Hof van den Keijzer" hebben we "extraordinaris envoyé" al herkend, maar we moeten ook "keizer" vinden. Deze stap zorgt ervoor dat we alle hoedanigheden in de string identificeren en annoteren.

Stap 3a: resultatenframe "0" doorzoeken, dus de onderdelen van de entiteiten vóór de herkende string.

In [ ]:
import pandas as pd
import re
import kws_hoe
import inspect
from tqdm import tqdm

def mark_pat(df, field, kw_dict, target_name):
    results = []

    # Compile all patterns beforehand
    compiled_patterns = {}
    for pattern, replacement in kw_dict.items():
        pattern_with_group = f"(.*) *(\\b{pattern}\\b) *(.*)"
        compiled_patterns[pattern] = {
            'compiled_pattern': re.compile(pattern_with_group, flags=re.I),
            'replacement': replacement
        }
    
    # Apply annotations using the compiled patterns with a progress bar
    for pattern, data in tqdm(compiled_patterns.items(), desc="Matches zoeken", unit="pattern"):
        ptrn = data['compiled_pattern']
        replacement = data['replacement']
        
        matches = df[field].str.extractall(ptrn)
        if not matches.empty:
            matches['provenance'] = pattern
            matches[target_name] = replacement
            results.append(matches)

    if results:
        offpat = pd.concat(results)
        offpat = offpat.sort_values('provenance')
        
        # Rename columns based on target_name
        if target_name == 'anno_name0':
            offpat = offpat.rename(columns={0: 'prov_0'})
        elif target_name == 'anno_name2':
            offpat = offpat.rename(columns={2: 'prov_2'})
        elif target_name == 'anno_loc':
            offpat = offpat.rename(columns={0: 'prov_loc'})  # Adjust column index as needed
        elif target_name == 'anno_qual':
            offpat = offpat.rename(columns={2: 'prov_qual'})  # Adjust column index as needed
        
        offpat['old_ind'] = offpat.index.get_level_values(0)  # Keep old indices

    else:
        offpat = pd.DataFrame(columns=[0, 1, 2, 'provenance', target_name])

    return offpat

# Get all dictionaries from kws_hoe module
kw_names = {name: value for name, value in inspect.getmembers(kws_hoe, lambda x: isinstance(x, dict)) if not name.startswith("__") and name not in ["kw_post", "kw_pre", "kw_mid"]}


In [ ]:
import re
import pandas as pd
new_df.index = [idx[0] if isinstance(idx, tuple) else idx for idx in new_df.index]
new_df = new_df.rename(columns={'provenance': 'prov_1'})

preresults = {}
qual_results = {}
loc_results = {}

for kw in kw_names:
    print("Kws zoeken:", kw)
    
    if kw == 'kws_qual':
        target_col = 'anno_qual'
    elif kw == 'kws_loc':
        target_col = 'anno_loc'
    else:
        target_col = 'anno_0'
    
    offpat = mark_pat(new_df, field=0, kw_dict=kw_names[kw], target_name=target_col)
    print("Aantal gevonden voor", kw, ":", len(offpat))

    if len(offpat) > 0:
        if kw == 'kws_qual':
            qual_results[kw] = offpat
        elif kw == 'kws_loc':
            loc_results[kw] = offpat
        else:
            preresults[kw] = offpat

if preresults:
    preresult_tot = pd.concat(preresults)
    print("Totaal gevonden (exclusief qual):", len(preresult_tot))
else:
    preresult_tot = pd.DataFrame(columns=['index', '0'])
    print("Niets gevonden voor non-qual keywords.")

if qual_results:
    qual_result_tot = pd.concat(qual_results)
    print("Totaal gevonden voor qual:", len(qual_result_tot))
else:
    qual_result_tot = pd.DataFrame(columns=['index', '0'])
    print("Niets gevonden voor qual.")

if loc_results:
    loc_result_tot = pd.concat(loc_results)
    print("Totaal gevonden voor loc:", len(loc_result_tot))
else:
    loc_result_tot = pd.DataFrame(columns=['index', '0'])
    print("Niets gevonden voor loc.")

def ensure_dataframe(df, new_col_name):
    if isinstance(df, pd.Series):
        df = df.to_frame()
    if df.empty:
        df = pd.DataFrame(columns=[new_col_name])
    else:
        df.columns = [new_col_name] + df.columns.tolist()[1:]
    return df

preresult_tot = ensure_dataframe(preresult_tot, '0')
qual_result_tot = ensure_dataframe(qual_result_tot, '0')
loc_result_tot = ensure_dataframe(loc_result_tot, '0')


Nu voegen we de resultaten van deze zoektocht samen met het resultatenframe (new_df).

In [ ]:
import numpy as np
import pandas as pd

# Step 1: Initialize new columns in new_df
new_df['anno_0'] = np.nan
new_df['anno_loc'] = np.nan
new_df['anno_qual'] = np.nan
new_df['prov_0'] = np.nan
new_df['prov_qual'] = np.nan
new_df['prov_loc'] = np.nan


# Step 2: Function to merge and concatenate annotations, ensuring no duplicates
def merge_annotations(src_df, target_df, src_col, tgt_col):
    # Create a dictionary to hold sets of aggregated values for uniqueness
    temp_dict = {}
    for idx, row in src_df.iterrows():
        old_ind = row['old_ind']
        value = row[src_col]
        if old_ind in temp_dict:
            temp_dict[old_ind].add(value)
        else:
            temp_dict[old_ind] = {value}

    # Update target_df based on the aggregated dictionary
    for index, values in temp_dict.items():
        new_value = ', '.join(sorted(values))
        if index in target_df.index:
            existing_value = target_df.at[index, tgt_col]
            if pd.isna(existing_value):
                target_df.at[index, tgt_col] = new_value
            else:
                # Combine existing and new values, avoid duplicates
                combined_values = set(existing_value.split(', ')) | values
                target_df.at[index, tgt_col] = ', '.join(sorted(combined_values))

# Step 3: Apply the function to each DataFrame
merge_annotations(preresult_tot, new_df, 'anno_0', 'anno_0')
merge_annotations(preresult_tot, new_df, 'provenance', 'prov_0')
merge_annotations(qual_result_tot, new_df, 'anno_qual', 'anno_qual')
merge_annotations(qual_result_tot, new_df, 'provenance', 'prov_qual')
merge_annotations(loc_result_tot, new_df, 'anno_loc', 'anno_loc')
merge_annotations(loc_result_tot, new_df, 'provenance', 'prov_loc')

# Printing or returning new_df to see the updates
#new_df

Stap 3b: "2" doorzoeken, dus de onderdelen van de entiteiten ná de herkende string.

In [ ]:
#ongeveer 20 seconden
postresults = {}
qual_results = {}
loc_results = {}

for kw in kw_names:
    print("Kws zoeken:", kw)
    
    if kw == 'kws_qual':
        target_col = 'anno_qual_2'
    elif kw == 'kws_loc':
        target_col = 'anno_loc_2'
    else:
        target_col = 'anno_2'
    
    offpat = mark_pat(new_df, field=2, kw_dict=kw_names[kw], target_name=target_col)
    print("Aantal gevonden voor", kw, ":", len(offpat))

    if len(offpat) > 0:
        if kw == 'kws_qual':
            qual_results[kw] = offpat
        elif kw == 'kws_loc':
            loc_results[kw] = offpat
        else:
            postresults[kw] = offpat

if postresults:
    postresult_tot = pd.concat(postresults)
    print("Totaal gevonden (exclusief qual):", len(postresult_tot))
else:
    postresult_tot = pd.DataFrame(columns=['index', '2'])
    print("Niets gevonden voor non-qual keywords.")

if qual_results:
    qual_result_tot = pd.concat(qual_results)
    print("Totaal gevonden voor qual:", len(qual_result_tot))
else:
    qual_result_tot = pd.DataFrame(columns=['index', '2'])
    print("Niets gevonden voor qual.")

if loc_results:
    loc_result_tot = pd.concat(loc_results)
    print("Totaal gevonden voor loc:", len(loc_result_tot))
else:
    loc_result_tot = pd.DataFrame(columns=['index', '2'])
    print("Niets gevonden voor loc.")

def ensure_dataframe(df, new_col_name):
    if isinstance(df, pd.Series):
        df = df.to_frame()
    if df.empty:
        df = pd.DataFrame(columns=[new_col_name])
    else:
        df.columns = [new_col_name] + df.columns.tolist()[1:]
    return df

preresult_tot = ensure_dataframe(preresult_tot, '2')
postresult_tot = ensure_dataframe(postresult_tot, '2')
qual_result_tot = ensure_dataframe(qual_result_tot, '2')
loc_result_tot = ensure_dataframe(loc_result_tot, '2')


In [ ]:
import numpy as np
import pandas as pd

# Nieuwe kolommen aanmaken
new_df['anno_2'] = np.nan  
new_df['anno_qual_2'] = np.nan  
new_df['anno_loc_2'] = np.nan  
new_df['prov_qual_2'] = np.nan  
new_df['prov_loc_2'] = np.nan 
new_df['prov_2'] = np.nan  

# Enhanced function to merge and concatenate annotations, ensuring no duplicates
def merge_annotations(src_df, target_df, src_col, tgt_col):
    # Create a dictionary to hold sets of aggregated values for uniqueness
    temp_dict = {}
    for idx, row in src_df.iterrows():
        old_ind = row['old_ind']
        value = row[src_col]
        if old_ind in temp_dict:
            temp_dict[old_ind].add(value)
        else:
            temp_dict[old_ind] = {value}

    # Update target_df based on the aggregated dictionary
    for index, values in temp_dict.items():
        if index in target_df.index:
            existing_value = target_df.at[index, tgt_col]
            if pd.isna(existing_value):
                # Directly assign if no existing value
                target_df.at[index, tgt_col] = ', '.join(sorted(values))
            else:
                # Split existing values, combine with new, ensure uniqueness, then join
                existing_set = set(existing_value.split(', '))
                combined_values = existing_set | values
                target_df.at[index, tgt_col] = ', '.join(sorted(combined_values))

# Apply the function to each DataFrame
merge_annotations(postresult_tot, new_df, 'anno_2', 'anno_2')
merge_annotations(postresult_tot, new_df, 'provenance', 'prov_2')
merge_annotations(qual_result_tot, new_df, 'anno_qual_2', 'anno_qual_2')
merge_annotations(qual_result_tot, new_df, 'provenance', 'prov_qual_2')
merge_annotations(loc_result_tot, new_df, 'anno_loc_2', 'anno_loc_2')
merge_annotations(loc_result_tot, new_df, 'provenance', 'prov_loc_2')


# Tussentijds opslaan
#new_df.to_pickle('new_dfstaart2.pkl')

Nu hebben we alle verwijzingen naar hoedanigheden thuisgebracht en geannoteerd. De volgende stap is overkoepelende categorieën toekennen aan de verwijzingen op basis van een handmatig samengestelde lijst. 
Deze categorieën [LICHT TOE: welke categorieën zijn het, welk doel hebben ze in de uiteindelijke applicatie?]

In [ ]:
cats_df = pd.read_csv('cats.tsv', sep='\t', header=0) #Handmatige lijst met categorie-toewijzingen inlezen.

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Step 1: Ruimte maken voor de categorielabels
new_df['cat1'] = np.nan
new_df['cat0'] = np.nan
new_df['cat2'] = np.nan

# Step 2: Zoeken
def label_column_with_priority(row, anno_column):
    if pd.isna(row[anno_column]):
        return np.nan

    # Check for a match in the 'Rood' column
    match = cats_df.loc[cats_df['Rood'] == row[anno_column], 'Cat_1_nieuw']
    if not match.empty:
        return match.iloc[0]  # Return the first match

    # If no match in 'Rood', check for a match in the 'Hoedanigheid' column
    match = cats_df.loc[cats_df['Hoedanigheid'] == row[anno_column], 'Cat_1_nieuw']
    if not match.empty:
        return match.iloc[0]  # Return the first match

    # If no match is found, return 'Rest'
    return 'Rest'

# Apply the function with a progress bar
tqdm.pandas(desc="Categorieën toekennen")
new_df['cat1'] = new_df.progress_apply(lambda row: label_column_with_priority(row, 'anno_name1'), axis=1)
new_df['cat0'] = new_df.progress_apply(lambda row: label_column_with_priority(row, 'anno_0'), axis=1)
new_df['cat2'] = new_df.progress_apply(lambda row: label_column_with_priority(row, 'anno_2'), axis=1)

# Save the updated DataFrame
#new_df.to_pickle('new_dffull.pkl')
#new_df.to_csv('updated_data.tsv', sep='\t', index=False)


Het resultatenframe `new_df` is eenvoudig van opzet: het bevat de genormaliseerde strings (`lowertag`), opgesplitst in drie onderdelen (0, 1, 2), en de toegekende annotaties (`anno_name1-5`) en reguliere expressies (`provenance`). In de volgende stap combineren we het geaggregeerde frame `df` met de resultaten uit `new_df`, zodat alle informatie (inclusief de oorspronkelijke tag, aantallen, en nog niet herkende strings) op één centrale plek staat.

In [ ]:
#new_df = pd.read_pickle('new_dffull.pkl') #eventuele stap als je hierboven het resultatenframe hebt opgeslagen en nu weer hier wilt beginnen 

In [ ]:
import numpy as np
df['cat0'] = np.nan
df['cat1'] = np.nan
df['cat2'] = np.nan
df['provenance'] = np.nan
df['prov_0'] = np.nan
df['prov_1'] = np.nan
df['prov_2'] = np.nan
df['prov_loc'] = np.nan
df['anno_qual'] = np.nan
df['anno_loc'] = np.nan
df['anno_qual_2'] = np.nan
df['anno_loc_2'] = np.nan
df['prov_qual_2'] = np.nan
df['prov_loc_2'] = np.nan
df['prov_2'] = np.nan
df['anno_name1'] = new_df['anno_name1'] #where the magic happens
df['anno_loc'] = new_df['anno_loc'] #where the magic happens
df['anno_loc_2'] = new_df['anno_loc_2'] #where the magic happens
df['anno_qual'] = new_df['anno_qual'] #where the magic happens
df['anno_qual_2'] = new_df['anno_qual_2'] #where the magic happens
df['anno_name3'] = new_df['anno_0'] #where the magic happens
df['anno_name2'] = new_df['anno_2'] #where the magic happens
df['cat0'] = new_df['cat0']
df['cat2'] = new_df['cat2'] #where the magic happens
df['cat1'] = new_df['cat1'] 
df['prov_loc'] = new_df['prov_loc']
df['prov_0'] = new_df['prov_0']
df['prov_1'] = new_df['prov_1']
df['prov_2'] = new_df['prov_2']

df_delete = df[df['delete'] == 'x']
df_zonder_delete = df[~(df['delete'] == 'x')]
df_matched = df_zonder_delete[df_zonder_delete[['anno_name1', 'anno_name2', 'anno_name3', 'anno_name4']].notna().any(axis=1)]
df_unmatched = df_zonder_delete[df_zonder_delete[anno_columns].isna().all(axis=1) & ~df['delete'].astype(str).eq('x')]
df_matched.to_csv('sep.tsv',sep='\t')


Optionele tussenstap: bereken welk percentage van de verwijzingen tags heeft gekregen. Dit percentage is relatief aan het vooraf gespecificeerde minimumaantal van voorkomens, in ons geval 10 (zie bovenaan).

In [ ]:
total_entities = df['aantal'].sum()
total_matched_entities = df_matched['aantal'].sum() + df_delete['aantal'].sum()
total_unmatched_entities = df_unmatched['aantal'].sum()

# percentage berekenen
if total_entities > 0:
    percentage_matched = (total_matched_entities / total_entities) * 100
else:
    percentage_matched = 0

print(f"Hoeveelheid matches: {percentage_matched:.2f}%")


In [ ]:
df_matched.head(10)

We hebben nu een overzicht van alle door de NER-herkende verwijzingen naar hoedanigheden, en bijbehorende tags die hierbij horen. 

Ons doel is om in de Goetgevonden-applicatie de tags direct in de tekst weer te geven. Hiervoor moeten we ervoor zorgen dat elke verwijzing een exacte locatie in de tekst heeft, zodat gebruikers op de tekst kunnen klikken en de bijbehorende tag zien. De oorspronkelijke output van de NER-tagger, waarmee we dit notebook hebben geopend, bevat deze informatie.

Nu willen we de geaggregeerde tag-informatie terugzetten in dit bestand. Het eindresultaat van dit notebook is om verwijzingen naar hoedanigheden te verrijken met standaardnamen. Deze hoedanigheden moeten genest worden weergegeven (dat wil zeggen, met meerdere, soms overlappende hoedanigheden in de tekst waar relevant) en gecategoriseerd worden in overkoepelende groepen zoals adel & vorstenhuizen, politiek & bestuur, enzovoort.

De volgende stappen zijn gericht op het koppelen van de verzamelde informatie aan de oorspronkelijke tekstverwijzingen, inclusief hun exacte locatie in de tekst.

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Ensure that both columns are strings and strip any leading/trailing whitespace
annotations['tag_text'] = annotations['tag_text'].astype(str).str.strip()
df_matched['tag'] = df_matched['tag'].astype(str).str.strip()

# Perform the merge, keeping all columns from both DataFrames
annotations = annotations.merge(df_matched, left_on='tag_text', right_on='tag', how='left', suffixes=('', '_matched'))

# Check if the merge has resulted in NaNs and if there are indeed matching values
if annotations.isna().all().all():
    print("Some columns have only NaN values after merging.")
    # Optional: Debug by checking a few mismatches
    print("Examples of mismatched rows:")
    print(annotations[annotations['tag'].isna()].head())

# Display the first 10 rows of the merged DataFrame
annotations.head(10)


In [ ]:

annotations = pd.read_csv('./annotations-layer-HOE.tsv', sep='\t')

# Apply regex to clean 'tag_text' and create 'tag_new' column
annotations['tag_new'] = annotations['tag_text'].str.replace(r'^[^a-zA-Z0-9]+', '', regex=True)

# Define the columns that need to be copied from df_matched to annotations
columns_to_add = [
    'lowertag', 'anno_name1', 'anno_name2', 'anno_name3', 'anno_name4', 'anno_name5',
    'delete', 'best_score', 'select', 'uitzoeken/onduidelijk', 'anno_loc', 
    'anno_qual', 'cat0', 'cat1', 'cat2', 'provenance', 'prov_0', 'prov_1', 
    'prov_2', 'prov_loc', 'anno_qual_2', 'anno_loc_2', 'prov_qual_2', 'prov_loc_2'
]

# Ensure all required columns are present in annotations
for col in columns_to_add:
    if col not in annotations.columns:
        annotations[col] = np.nan

# Normalize 'tag_text' in annotations and 'tag' in df_matched
annotations['tag_text'] = annotations['tag_text'].str.strip().str.lower()
df_matched['tag'] = df_matched['tag'].str.strip().str.lower()

# Remove duplicates in df_matched by keeping the first occurrence of each tag
df_matched_unique = df_matched.drop_duplicates(subset='tag', keep='first')

# Merge the two DataFrames to add columns from df_matched to annotations
# Use a left merge to retain all rows from annotations
annotations = annotations.merge(df_matched_unique, left_on='tag_text', right_on='tag', how='left', suffixes=('', '_matched'))

# Rename and reorder columns as needed
for col in columns_to_add:
    if col in annotations.columns:
        annotations[col] = annotations[f'{col}_matched']

# Drop intermediate columns
annotations = annotations.drop(columns=[f'{col}_matched' for col in columns_to_add if f'{col}_matched' in annotations.columns])

# Sort the annotations DataFrame by 'anno_name1'
annotations = annotations.sort_values('anno_name1')

# Display the first 100 rows of the updated annotations DataFrame
annotations.head(100)


In [ ]:
total_rows = len(annotations)

# Calculate the number of rows where 'anno_name1' is not empty or NaN
non_empty_anno_name1 = annotations['anno_name1'].notna() & (annotations['anno_name1'].str.strip() != '')

# Calculate the percentage of rows with non-empty 'anno_name1'
percentage_non_empty = non_empty_anno_name1.sum() / total_rows * 100

# Print the result
print(f"Percentage of annotations with non-empty 'anno_name1': {percentage_non_empty:.2f}%")


Het eindresultaat van dit notebook willen we exporteren als JSON, waarin de tekstverwijzingen samen worden gevoegd met de standaardversies van hoedanigheden en overkoepelende categorieën.  

In dit project hebben we twee versies van JSON-bestanden gemaakt om de annotaties van hoedanigheden te representeren. Deze annotaties kunnen kwalificaties en locaties bevatten die aan een entiteit zijn gekoppeld. De kwalificaties en locaties worden op twee manieren verwerkt:

Met kwalificaties en locaties tussen haakjes:

In deze versie worden kwalificaties en locaties toegevoegd aan de naam van de entiteit, tussen haakjes. Bijvoorbeeld:
"plenipotentiaris (vice)"
"generaal (engels)"
Deze aanpak maakt het mogelijk om extra context of specificatie direct bij de naam van de entiteit te voegen, wat helpt bij het verduidelijken van de rol of functie van de entiteit binnen het document.

Zonder kwalificaties en locaties tussen haakjes:

In deze versie worden kwalificaties en locaties niet tussen haakjes toegevoegd aan de naam van de entiteit. Alleen de basisnaam van de entiteit wordt opgenomen.
Dit kan nuttig zijn wanneer je een eenvoudiger formaat nodig hebt zonder extra context, of wanneer de kwalificaties en locaties niet van belang zijn voor je toepassing.



In [ ]:
import pandas as pd
import json
from tqdm import tqdm

# Assuming annotations and anno_columns are already defined
anno_columns = ['anno_name1', 'anno_name2', 'anno_name3', 'anno_name4', 'anno_name5',
                'anno_loc', 'anno_loc_2', 'anno_qual', 'anno_qual_2']
cat_columns = ['cat1', 'cat2', 'cat0', 'cat0', 'cat0']  # cat0 mapped to anno_name3

annotations_matched = annotations

# Update create_json_objects function to include category labels and provenance
def create_json_objects(row):
    json_objects = []
    for col, cat_col in zip(anno_columns, cat_columns):
        if pd.notnull(row[col]):
            category_label = row[cat_col]
            provenance = None

            # Modify entity name and provenance for qualifiers and locations
            if col == 'anno_qual' and pd.notnull(row['anno_name1']):
                entity_name = f"{row['anno_name1']} ({row['anno_qual']})"
                provenance = f"{row['prov_1']}, {row['prov_qual']}"
            elif col == 'anno_qual_2':
                if pd.notnull(row['anno_name2']):
                    entity_name = f"{row['anno_name2']} ({row['anno_qual_2']})"
                    provenance = f"{row['prov_2']}, {row['prov_qual_2']}"
                else:
                    entity_name = f"{row['anno_name1']} ({row['anno_qual_2']})"
                    provenance = f"{row['prov_1']}, {row['prov_qual_2']}"
            elif col == 'anno_loc' and pd.notnull(row['anno_name1']):
                entity_name = f"{row['anno_name1']} ({row['anno_loc']})"
                provenance = f"{row['prov_1']}, {row['prov_loc']}"
            elif col == 'anno_loc_2':
                if pd.notnull(row['anno_name2']):
                    entity_name = f"{row['anno_name2']} ({row['anno_loc_2']})"
                    provenance = f"{row['prov_2']}, {row['prov_loc_2']}"
                else:
                    entity_name = f"{row['anno_name1']} ({row['anno_loc_2']})"
                    provenance = f"{row['prov_1']}, {row['prov_loc_2']}"
            else:
                entity_name = row[col]
                if col == 'anno_name1':
                    provenance = row['prov_1']
                elif col == 'anno_name2':
                    provenance = row['prov_2']
                elif col == 'anno_name3':
                    provenance = row['prov_0']
            
            json_object = {
                "entity": {
                    "name": entity_name,
                    "category": "HOE",
                    "labels": [category_label] if pd.notnull(category_label) else []
                },
                "reference": {
                    "layer": row['layer'],
                    "inv": row['inv'],
                    "tag_text": row['tag_text'],
                    "resolution_id": row['resolution_id'],
                    "paragraph_id": row['paragraph_id'],
                    "offset": row['offset'],
                    "end": row['end'],
                    "tag_length": row['tag_length']
                }
            }
            if provenance:
                json_object['reference']['provenance'] = [ provenance ]
            
            json_objects.append(json_object)
    return json_objects

# Generate JSON objects
json_data_list_of_lists = []
for index, row in tqdm(annotations_matched.iterrows(), total=len(annotations_matched), desc='Creating JSON objects'):
    json_data_list_of_lists.extend(create_json_objects(row))

# Save JSON to file
with open('annotations_matched_qualloc.json', 'w') as json_file:
    json.dump(json_data_list_of_lists, json_file, indent=4)

print('annotations_matched_qualloc.json')


Json2: zonder loc en qual tussen haakjes

In [ ]:
import pandas as pd
import json
from tqdm import tqdm

# zonder loc en qual
anno_columns = ['anno_name1', 'anno_name2', 'anno_name3', 'anno_name4', 'anno_name5']
cat_columns = ['cat1', 'cat2', 'cat0', 'cat0', 'cat0']  # cat0 mapped to anno_name3

annotations_matched = annotations[~annotations[anno_columns].isna().all(axis=1)]

def create_json_objects(row):
    json_objects = []
    for col, cat_col in zip(anno_columns, cat_columns):
        if pd.notnull(row[col]):
            category_label = row[cat_col]
            provenance = None

            entity_name = row[col]
            if col == 'anno_name1':
                provenance = row['prov_1']
            elif col == 'anno_name2':
                provenance = row['prov_2']
            elif col == 'anno_name3':
                provenance = row['prov_0']

            json_object = {
                "entity": {
                    "name": entity_name,
                    "category": "HOE",
                    "labels": [category_label] if pd.notnull(category_label) else []
                },
                "reference": {
                    "layer": row['layer'],
                    "inv": row['inv'],
                    "tag_text": row['tag_text'],
                    "resolution_id": row['resolution_id'],
                    "paragraph_id": row['paragraph_id'],
                    "offset": row['offset'],
                    "end": row['end'],
                    "tag_length": row['tag_length']
                }
            }
            if provenance:
                json_object['reference']['provenance'] = [ provenance ]
            
            json_objects.append(json_object)
    return json_objects

json_data_list_of_lists = []
for index, row in tqdm(annotations_matched.iterrows(), total=len(annotations_matched), desc='Creating JSON objects'):
    json_data_list_of_lists.extend(create_json_objects(row))

with open('annotations_matched_simple.json', 'w') as json_file:
    json.dump(json_data_list_of_lists, json_file, indent=4)

print('annotations_matched_simple.json')


Finally, we export a list of matched rows for use by the name recognition notebook.

In [ ]:
def is_matched(cols):
    return any([x==x for x in list(cols)])
annotations['matched'] = annotations[anno_columns].apply(is_matched, axis=1)

In [ ]:
annotations[['layer','resolution_id','paragraph_id','offset','end','matched']
].to_csv('matched_hoedanigheden.tsv', sep='\t', index=False)